In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

from InteriorHelmholtzSolver2D import *
from ExampleBoundaries import Square

c         = 344.0 # speed of sound [m/s]
rho       = 1.205 # density of air [kg/m^3]
frequency = 400.0 # frequency [Hz]


# Test Problem 1
# Dirichlet boundary condition with phi = sin(k/sqrt(2)*x) * sin(k/sqrt(2)*y)
#
solver = InteriorHelmholtzSolver2D(*(Square()))

boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)

k = frequencyToWavenumber(frequency)

boundaryCondition.f[:] = np.sin(k/np.sqrt(2.0) * solver.aCenters[:,0]) \
                       * np.sin(k/np.sqrt(2.0) * solver.aCenters[:,1])

boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorPoints = np.array([[0.0250, 0.0250],
                           [0.0750, 0.0250],
                           [0.0250, 0.0750],
                           [0.0750, 0.0750],
                           [0.0500, 0.0500]], dtype=np.float32)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# von Neumann boundary condition such that phi = sin(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# Differentiate with respect to x and y to obtain outward normal:
# dPhi/dX = k/sqrt(2) * cos(k/sqrt(2) * x) * sin(k/sqrt(2) * y)
# dPhi/dY = k/sqrt(2) * sin(k/sqrt(2) * x) * cos(k/sqrt(2) * y)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
c = k / np.sqrt(2.0)
for i in range(solver.aCenters.shape[0]):
    x = solver.aCenters[i, 0]
    y = solver.aCenters[i, 1]
    if (x < 1e-7):
        boundaryCondition.f[i] = -c * np.cos(c * x) * np.sin(c * y)
    elif (x > 0.1 - 1e-7):
        boundaryCondition.f[i] =  c * np.cos(c * x) * np.sin(c * y)
    elif (y < 1e-7):
        boundaryCondition.f[i] = -c * np.sin(c * x) * np.cos(c * y)
    else:
        boundaryCondition.f[i] =  c * np.sin(c * x) * np.cos(c * y)        

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)

   
# Test Problem 3
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The rigid boundary implies the bondary condition v=0.
# The test problem computes the field produced by a unit source at
# the point (0.5,0.25) within the square with a rigid boundary.
# The incident velocity potential is given by {\phi}_inc=i*h0(kr)/4
# where r is the distance from the point (0.5,0.25)
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

p = np.array([0.05, 0.025], dtype=np.float32)
for i in range(solver.aCenters.shape[0]):
    r = solver.aCenters[i] - p
    R = norm(r)
    boundaryIncidence.phi[i] = 0.25j * hankel1(0, k * R)
    if solver.aCenters[i, 0] < 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[0] / R)
    elif solver.aCenters[i, 0] > 0.1 - 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[0] / R)
    elif solver.aCenters[i, 1] < 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * (-r[1] / R)
    elif solver.aCenters[i, 1] > 0.1 - 1e-7:
        boundaryIncidence.v[i] = -0.25j * k * hankel1(1, k * R) * ( r[1] / R)
    else:
        assert False, "All cases must be handled above."
        
for i in range(interiorIncidentPhi.size):
    r = interiorPoints[i] - p
    R = norm(r)
    interiorIncidentPhi[i] = 0.25j * hankel1(0, k * R)
       
boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)
interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)
print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)